## 1. Branch & Bound方法

本notebook对应[这一篇blog post](https://hanqiu92.github.io/blogs/2020/IP_BB_solver_1_202006/)中的内容，主要包括对分支定界法（Branch & Bound, B&B）的基本实现。

首先是准备工作：导入相关的计算工具包、并定义一些基础类型和常数。

In [1]:
import time
import math
import numpy as np
from cylp.cy import CyClpSimplex
np.set_printoptions(suppress=True,precision=4)

#############################
## 一些与数值误差相关的常数的定义
INF = 1e16 ## infinity
MAX = 1e20 ## 默认最大值，比infinity稍大
TOL = 1e-6 ## 数值误差tolerance
#############################

from enum import Enum,unique
#############################
## 用于记录状态的枚举类型
@unique
class SolveStatus(Enum):
    ## 求解状态
    ONGOING = 0 ## 求解进行中
    OPT = 1 ## 最优解
    INFEAS = 2 ## 不可行
    ERR = -1 ## error
#############################

在blog中，我介绍了以下B&B方法求解框架：

1. 根据原始问题生成根结点$N_0=R$；
2. 初始化问题结点队列$\mathcal{L} = \{N_0\}$以及全局解$x=\emptyset$和目标值上下界$\hat{c},\check{c}=\infty$；
3. (Select) 从队列$\mathcal{L}$中选取一个问题结点$N_j$，并令问题队列$\mathcal{L}=\mathcal{L}/\{N_j\}$；
4. (Solve) 调用LP求解器求解问题$N_j$；如果问题可行，则可得最优解$x_j$和目标值$\check{c}_j$（作为下界估计）；否则，令$x_j=\emptyset,\ \check{c}_j = \infty$；
5. (Process) 根据下界估计$\check{c}_j$，对B&B树中结点$N_j$与根结点$N_0$路径上相关结点$N_{j'}$的下界信息$\check{c}_{j'}$进行更新，然后进行以下判断：
   1. 如果$\check{c}_j$不小于全局最优目标值/上界$\hat{c}$，则可以直接忽略该结点；
   2. 如果$x_j$整数可行且$\check{c}_j < \hat{c}$，则更新全局解$x \gets x_j$和$\hat{c} \gets \check{c}_j$；
   3. (Branch) 否则，根据$x_j$的情况，将问题$N_j$拆分成子问题结点$N_{j_1},\cdots,N_{j_n}$，并将它们加入队列$\mathcal{L}$中。
6. 如果队列$\mathcal{L}$为空，返回$x,\hat{c},\check{c}$；否则，回到第3步。

接下来，我将通过**类BBSolver**来实现上述求解框架，并通过**类Node**对子问题/结点的信息（包括子问题的变量上下界等）进行管理。进一步地，为了后续可灵活扩展，我将引入**Selector类**和**Brancher类**来分别对node selection和branching阶段的工作进行封装。在实现过程中，需要特别注意以下两个问题：

1. 内存管理。由于B&B流程中可能产生指数多个子问题/结点，如果每个结点都存储大量的信息，则算法可能很快就会耗尽可用内存。因此，有必要对各结点处的局部信息进行生命周期管理，及时清理不再使用的变量来释放存储空间。特别地，在这里的实现中，我参考了SCIP中的做法，将每个结点的相关计算集中起来，并通过activate/deactivate等方法来动态生成和清理这些计算所依赖的局部信息。
2. 计算效率。在B&B流程中，有些工作需要遍历各结点（例如node selection阶段）；如果能对一些后续会重复使用的信息进行全局管理，则能有效提高计算效率。例如，在下面的代码中，我在BBSolver类中引入了unprocessed_node_bounds来管理所有未处理结点的目标值下界信息。

具体的实现细节可参见下面代码中的注释。

In [2]:
import heapq

class Node:
    '''
    B&B树中的结点，对应一个bound tightened的子问题。主要负责管理以下信息：
        作为树的一个元素，需要存储指向父节点和子节点的指针以及自身的idx、深度。
        作为原问题的一个子问题，需要存储相比原问题的modification。
        作为一个MIP问题，需存储解的相关信息：obj、sol等。
        为了B&B整体流程的效果评估，需要存储到目前为止子树的目标值上下界。
    '''
    
    def __init__(self,node_idx=0,parent=None,
                 var_idx=-1,var_sol=0,var_bnd=0,
                 var_lb=0,var_ub=0,init_var_lb=0,init_var_ub=0):
        ## 在B&B树中的位置及状态
        self.idx = node_idx
        self.parent,self.childs = parent,[]
        self.level = 0
        if parent is not None:
            self.level = parent.level + 1
        self.is_processed,self.is_active = False,False

        ## 定义子问题所需的边界信息
        self.var_idx = var_idx
        self.var_bnd = var_bnd
        self.var_delta = var_bnd - var_sol
        if parent is not None:
            self.local_vars_lb_dict,self.local_vars_ub_dict = \
                parent.local_vars_lb_dict.copy(),parent.local_vars_ub_dict.copy()
            if self.var_delta > 0:
                self.local_vars_lb_dict[(var_idx,init_var_lb)] = var_lb ## new lb
            else:
                self.local_vars_ub_dict[(var_idx,init_var_ub)] = var_ub ## new ub
        else:
            self.local_vars_lb_dict,self.local_vars_ub_dict = dict(),dict()

        ## 其他求解相关信息
        self.LB = self.obj = MAX
        if parent is not None:
            self.LB = parent.obj ## 用父结点的求解目标值作为子结点的目标值下界估计
        self.branch_var_idx = -1
        self.branch_var_sol = 0
        
    def activate(self,solver):
        self.is_processed = True
        self.is_active = True
        self.sol,self.sol_feas = None,False

    def update(self,sol,obj,solver):
        ## 更新子问题的相关信息
        self.sol,self.obj,self.LB = sol,obj,obj
        sol_bool_infeas = (np.abs(sol - np.round(sol)) > TOL) & solver.int_vars_bool
        self.sol_feas = (~np.any(sol_bool_infeas))

    def deactivate(self):
        ## 删除中间变量，释放内存
        self.is_active = False
        del self.sol,self.sol_feas
        del self.local_vars_lb_dict,self.local_vars_ub_dict
        
    def clear(self):
        if self.is_active:
            self.deactivate()
        del self.parent,self.childs

class BBSolver:
    '''
    B&B求解流程
    '''
    def __init__(self,problem):
        '''
        B&B流程初始化
        
        输入
        problem: CyClpSimplex对象，定义了初始MIP问题
        '''
        ## 获取MIP问题的相关信息（通过CyLP的API）
        self.problem = problem
        self.int_vars_bool = problem.integerInformation.copy() > 0 ## 决策变量的整数信息
        self.int_vars_idx = np.where(self.int_vars_bool)[0] ## 整数决策变量的下标
        self.vars_ub = problem.variablesUpper ## 对问题的变量上界的reference
        self.vars_lb = problem.variablesLower ## 对问题的变量下界的reference
        self.num_vars = len(self.vars_ub) ## 问题的变量个数
        self.init_vars_ub = self.vars_ub.copy() ## 问题的原始变量上界
        self.init_vars_lb = self.vars_lb.copy() ## 问题的原始变量上界
        self.primal_sol = self.problem.primalVariableSolution ## 对问题的LP relaxation的最优primal解的reference
        
        ## 初始化各执行对象
        self.brancher = Brancher() ## 负责branching的对象
        self.selector = Selector() ## 负责node selection的对象
        
        ## 初始化B&B树的相关信息
        self.nodes = [] ## 生成结点队列
        self.num_nodes,self.next_node_idx = 0,0 ## 当前结点数和下一个结点下标
        self.unprocessed_node_idxs = set([]) ## 未处理结点下标
        self.unprocessed_node_bounds = [] ## 未处理结点的下界信息
        heapq.heapify(self.unprocessed_node_bounds)
        
        self.global_LB = MAX ## MIP问题目标值的全局下界
        self.global_obj,self.global_sol = MAX,None ## MIP问题的当前最优解和目标值
        
    def add_node(self,node):
        self.nodes.append(node)
        ## 更新未处理结点的信息
        self.unprocessed_node_idxs.add(node.idx)
        heapq.heappush(self.unprocessed_node_bounds,(node.LB,node.idx))
        self.num_nodes += 1
        self.next_node_idx += 1
        
    def modify_problem_bounds(self,node):
        '''
        根据Node中存储的子问题信息，对问题接口中的变量上下界进行修改
        '''
        for (var_idx,init_var_bnd),var_bnd in node.local_vars_lb_dict.items():
            self.vars_lb[var_idx] = var_bnd
        for (var_idx,init_var_bnd),var_bnd in node.local_vars_ub_dict.items():
            self.vars_ub[var_idx] = var_bnd
    
    def recover_problem_bounds(self,node):
        '''
        根据Node中存储的子问题信息，对问题接口中的变量上下界进行恢复
        '''
        for (var_idx,init_var_bnd),var_bnd in node.local_vars_lb_dict.items():
            self.vars_lb[var_idx] = init_var_bnd
        for (var_idx,init_var_bnd),var_bnd in node.local_vars_ub_dict.items():
            self.vars_ub[var_idx] = init_var_bnd
    
    def node_solve(self,node):
        '''
        Node中的子问题求解
        '''
        ## dual simplex求解LP relaxation，并获取相应解和目标值
        self.problem.dual() 
        sol = self.primal_sol.copy()
        obj = self.problem.objectiveValue if self.problem.getStatusCode() == 0 else MAX
        if node.level == 0:
            self.root_sol = sol.copy()
        
        ## 根据求解结果对node的信息进行更新
        node.update(sol,obj,self)
        
        ## 判断是否产生了更好的整数解
        if node.sol_feas and node.obj < self.global_obj:
            ## 更新全局可行解
            self.global_obj,self.global_sol = node.obj,node.sol.copy()        
            
    def create_node(self,parent_node,var_idx,var_lb,var_ub,var_sol,var_bnd):
        '''
        创建子结点
        '''
        new_node = Node(node_idx=self.next_node_idx,parent=parent_node,
                        var_idx=var_idx,var_lb=var_lb,var_ub=var_ub,var_sol=var_sol,var_bnd=var_bnd,
                        init_var_lb=self.init_vars_lb[var_idx],init_var_ub=self.init_vars_ub[var_idx])
        parent_node.childs += [new_node]
        self.add_node(new_node)
    
    def branch(self,node):
        '''
        branching
        '''
        ## 调用brancher的branch方法找到branch var idx
        var_idx = self.brancher.branch(self,node) 
        node.branch_var_idx,node.branch_var_sol = var_idx,node.sol[var_idx]
        
        ## 创建子结点
        x_j = node.sol[var_idx]
        u_j,l_j = self.vars_ub[var_idx],self.vars_lb[var_idx]
        u_j_new,l_j_new = math.floor(x_j),math.ceil(x_j)
        for (var_lb,var_ub,var_sol,var_bnd) in \
            [(l_j,u_j_new,x_j,u_j_new),(l_j_new,u_j,x_j,l_j_new)]:
            self.create_node(node,var_idx,var_lb,var_ub,var_sol,var_bnd)
    
    def node_process(self,node_idx):
        '''
        对选出的node/子问题进行处理
        '''
        if node_idx not in self.unprocessed_node_idxs:
            print(self.unprocessed_node_idxs,node_idx)
            
        node = self.nodes[node_idx]
        self.unprocessed_node_idxs.remove(node_idx)
        
        if node.LB >= self.global_obj:
            ## 可以直接剪枝
            node.is_processed = True
            prun_status = 0
            return prun_status
        
        node.activate(self) ## 激活node，生成局部信息
        self.modify_problem_bounds(node) ## 更新问题接口中的变量上下界
        self.node_solve(node) ## 求解子问题
        
        ## 更新BB树的下界信息
        node_child = node
        update_lb_flag = True
        while (node_child.level > 0) and update_lb_flag:
            node_parent = node_child.parent
            curr_LB = MAX
            for child in node_parent.childs:
                if child.LB < curr_LB:
                    curr_LB = child.LB
                if not child.is_processed:
                    ## no need to further update LB
                    update_lb_flag = False
                    break
            node_parent.LB = curr_LB
            node_child = node_parent  
        self.global_LB = self.root_node.LB
        
        prun_status = 0
        if not node.sol_feas and node.LB < self.global_obj:
            ## 子问题整数不可行且子问题最优解小于当前最优整数解，继续branch生成子问题
            self.branch(node)
            prun_status = 1
        
        self.recover_problem_bounds(node) ## 恢复问题接口中的变量上下界
        node.deactivate() ## 释放内存
        return prun_status
        
    def node_select(self,node_idx):
        '''
        node selection
        '''
        if len(self.unprocessed_node_idxs) == 0:
            ## 没有未处理的子问题
            return -1
        node = self.nodes[node_idx]
        ## 调用selector的select方法找到下一个node idx
        select_node_idx = self.selector.select(self,node)
        return select_node_idx

    def solve_root_node(self):
        ## 生成根结点
        root_node = Node()
        self.root_node = root_node
        self.add_node(root_node)
        ## 执行简化版的process
        self.unprocessed_node_idxs.remove(root_node.idx)
        root_node.activate(self) ## 激活node，生成局部信息
        self.node_solve(root_node) ## 求解子问题
        self.root_sol = root_node.sol ## 根结点（初始问题）的LP relaxation最优解
        self.global_LB = root_node.LB ## 更新BB树的上下界信息
        status = SolveStatus.ONGOING
        if not root_node.sol_feas and self.global_LB < self.global_obj:
            self.branch(root_node)
        else:
            status = SolveStatus.OPT
        root_node.deactivate() ## 释放内存
        return status
    
    def solve(self,time_limit=3600,iter_limit=1000000,if_disp=True):
        '''
        主求解流程
        '''
        tt = time.time()
        status = self.solve_root_node()
        if (status == SolveStatus.ONGOING):
            node_idx,iter_ = self.root_node.idx,0
            while (iter_ < iter_limit) and (time.time()-tt < time_limit):
                iter_ += 1
                node_idx = self.node_select(node_idx)
                if node_idx < 0:
                    ## 没有选出下一个子问题，报错退出
                    status = SolveStatus.ERR
                    break

                self.node_process(node_idx)
                if (self.global_obj - self.global_LB) < TOL:
                    ## 上下界差距小于tolerance，可认为找到最优解
                    status = SolveStatus.OPT
                    break

                if if_disp:
                    if iter_ % 100000 == 0:
                        print('iter {} LB {:.3e} UB {:.3e}'.format(iter_,self.global_LB,self.global_obj))
        LB,obj,dt = self.global_LB,self.global_obj,time.time()-tt
        return (dt,status,LB,obj)

在本notebook中，branching和node selection分别实现了最简单的版本：

* 在branching过程中，选出infeasibility最大（即距离整数最远）的变量作为branching变量；
* 在node selection过程中，选出目标值下界估计最小的node/子问题作为下一个要处理的node。

在下一个notebook中，我将介绍更多进阶的branching和node selection方法。

In [3]:
class Brancher:
    def __init__(self):
        self.branch = self.branch_inf_most

    def branch_inf_most(self,solver,node):
        ## 选出距离整数最远的变量
        score = np.abs(node.sol - np.round(node.sol))
        var_idx = solver.int_vars_idx[np.argmax(score[solver.int_vars_idx])]
        return var_idx

class Selector:
    def __init__(self):
        self.select = self.select_bfs

    def select_bfs(self,solver,curr_node):
        ## 选出下界
        node_idx = heapq.heappop(solver.unprocessed_node_bounds)[1]
        while (len(solver.unprocessed_node_bounds) > 0 and node_idx not in solver.unprocessed_node_idxs):
            node_idx = heapq.heappop(solver.unprocessed_node_bounds)[1]
        if node_idx not in solver.unprocessed_node_idxs:
            node_idx = -1
        return node_idx

下面，我们实现一个简单的求解入口，来对上述B&B实现进行测试。

In [4]:
from util import *
def bb_solve(model_name,time_limit=3600,if_disp=True):
    ## 读取问题
    s = CyClpSimplex()
    s.readMps(model2fname(model_name))
    s.logLevel = 0
    solver = BBSolver(s)
    dt,status,LB,obj = solver.solve(time_limit=time_limit,if_disp=if_disp)
    total_node_cnt,remain_node_cnt = len(solver.nodes),len(solver.unprocessed_node_idxs)
    for node in solver.nodes:
        node.clear()
    return dt,status,LB,obj,total_node_cnt,remain_node_cnt,solver

## 读取测试案例
with open('data/test_cases_v3','r+') as f:
    lines = f.readlines()
model_names = [line.strip('\n') for line in lines]

In [5]:
max_secs = 600 ## 每个问题限时10分钟
result = dict()
for model_name in model_names:
    dt,status,LB,obj,total_node_cnt,remain_node_cnt,_ = bb_solve(model_name,time_limit=max_secs,if_disp=False)
    print('{}: time={:.2f}, status={}. LB={:.3e}, obj={:.3e}. total nodes={}, remain nodes={}.'.format(
        model_name,dt,status,LB,obj,total_node_cnt,remain_node_cnt))
    result[model_name] = (dt,status,LB,obj,total_node_cnt,remain_node_cnt)

## 保存测试结果
df_result = process_result_new(result,save_fname='result/ours_benchmark_v1.csv')
print('elapsed hours: {:.2f}.'.format(df_result['time'].sum() / 3600))

binkar10_1: time=600.33, status=SolveStatus.ONGOING. LB=6.672e+03, obj=1.000e+20. total nodes=584629, remain nodes=291333.
dano3_3: time=40.95, status=SolveStatus.OPT. LB=5.763e+02, obj=5.763e+02. total nodes=33, remain nodes=16.
dano3_5: time=600.97, status=SolveStatus.ONGOING. LB=5.768e+02, obj=5.786e+02. total nodes=887, remain nodes=425.
eil33-2: time=139.10, status=SolveStatus.OPT. LB=9.340e+02, obj=9.340e+02. total nodes=9169, remain nodes=652.
gen-ip002: time=372.72, status=SolveStatus.ONGOING. LB=-4.801e+03, obj=1.000e+20. total nodes=1999957, remain nodes=999956.
gen-ip054: time=305.99, status=SolveStatus.ONGOING. LB=6.799e+03, obj=1.000e+20. total nodes=1983419, remain nodes=983418.
gmu-35-40: time=600.00, status=SolveStatus.ONGOING. LB=-2.407e+06, obj=1.000e+20. total nodes=1149161, remain nodes=574401.
gmu-35-50: time=600.00, status=SolveStatus.ONGOING. LB=-2.608e+06, obj=1.000e+20. total nodes=891301, remain nodes=445559.
icir97_tension: time=600.00, status=SolveStatus.ONG

通过上述测试我们可以看出，虽然上面的B&B实现能够执行，但对于一大部分测试问题，无法在时间限制内找到可行解；这与CBC求解器的B&B实现有很大差距。在下一个notebook中，我们将看到不同的branching和node selection方法对B&B方法求解结果的影响。